In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [2]:
import random
import json
from functools import reduce
from itertools import product
from operator import mul

In [3]:
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import time

from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, log_loss
from sklearn.model_selection import StratifiedKFold

In [13]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [5]:
train = pd.read_csv('_data/train_independent.csv')
test = pd.read_csv('_data/test_independent.csv')

In [6]:
ensemble_train = pd.read_csv('_data/ensemble_train_extra.csv')
ensemble_test = pd.read_csv('_data/ensemble_test_extra.csv')

In [16]:
cols = [col for col in ensemble_train.columns if 'PCA' in col or 'Kmeans' in col or 'SUBMOTIVO' in col]

In [19]:
anti_cols = [col for col in ensemble_train.columns if col not in cols]

In [18]:
train_extra = ensemble_train[cols]
test_extra = ensemble_test[cols]

In [21]:
ensemble_train = ensemble_train[anti_cols]
ensemble_test = ensemble_test[anti_cols]

## OOF LGBM

In [23]:
kfolds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

In [31]:
def run_model(model, X, y, dataset_desc='', params_desc=''):
    model_name = str(model.__class__).split('.')[-1].replace('>','').replace("'",'')
    print(model_name)
    predictions = y.copy()
    logloss = []
    for train_idx, test_idx in kfolds.split(X, y):
        model.fit(X.iloc[train_idx,:], y[train_idx])
        
        y_ = model.predict_proba(X.iloc[test_idx,:])[:,1]
        predictions[test_idx] = y_
        logloss += [log_loss(y.iloc[test_idx], y_)]
        print(logloss[-1])
        
    print(np.mean(logloss), np.std(logloss))
        
    return predictions

In [32]:
params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'num_leaves': 100,
    'reg_alpha': 1,
    'reg_lambda': 1,
    'colsample_bytree': 0.8,
}

In [ ]:
ensemble_train['LGBM-Extra'] = run_model(LGBMClassifier(**params), 
                    train_extra, train['ATTRITION'])

In [39]:
lgbm = LGBMClassifier(**params)
lgbm.fit(train_extra, train['ATTRITION'])
ensemble_test['LGBM-Extra'] = lgbm.predict_proba(test_extra)[:,1]

In [40]:
ensemble_train.shape, ensemble_test.shape

((70000, 19), (30000, 19))

In [45]:
train_extra.shape, test_extra.shape

((70000, 85), (30000, 85))

In [43]:
ensemble_train.to_csv('ensemble_train_input2.csv', index=False)
ensemble_test.to_csv('ensemble_test_input2.csv', index=False)

In [44]:
train_extra.to_csv('ensemble_train_input1B.csv', index=False)
test_extra.to_csv('ensemble_test_input1B.csv', index=False)

## Final Ensemble

In [54]:
train.shape, train_extra.shape, ensemble_train.shape

((70000, 73), (70000, 85), (70000, 19))

In [82]:
from keras.layers import Input, Dense, Activation, BatchNormalization, Dropout, concatenate
from keras.models import Model
from keras.optimizers import Adam

In [89]:
def create_model():
    main_input = Input((train.shape[1] + train_extra.shape[1] - 2,), name='main_input')
    layer1A = Dense(100)(main_input)
    layer1A = Activation('relu')(layer1A)
    layer1A = BatchNormalization()(layer1A)
    layer1A = Dropout(0.2)(layer1A)
    
    ensemble_input = Input((ensemble_train.shape[1] - 1,), name='ensemble_input')
    layer1B = Dense(12)(ensemble_input)
    layer1B = Activation('relu')(layer1B)
    layer1B = BatchNormalization()(layer1B)
    layer1B = Dropout(0.2)(layer1B)
    
    x = concatenate([layer1A, layer1B])
    
    x = Dense(100)(x)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    
    main_output = Dense(1, activation='sigmoid', name='main_output')(x)
    
    model = Model(inputs=[main_input, ensemble_input], outputs=[main_output])
    
    adam = Adam()
    model.compile(loss='binary_crossentropy', optimizer=adam)
    return model

In [91]:
model = create_model()

In [92]:
model.fit({
    'main_input': pd.concat((train.drop(['ID_CORRELATIVO', 'ATTRITION'], axis=1), 
                             train_extra), axis=1),
    'ensemble_input': ensemble_train.drop('ID_CORRELATIVO', axis=1)
    }, train['ATTRITION'], batch_size=32, epochs=50)

Epoch 1/50
70000/70000 [==============================] - 9s 132us/step - loss: 0.4116
Epoch 2/50
70000/70000 [==============================] - 9s 125us/step - loss: 0.3323
Epoch 3/50
70000/70000 [==============================] - 9s 126us/step - loss: 0.3209
Epoch 4/50
70000/70000 [==============================] - 9s 129us/step - loss: 0.3195
Epoch 5/50
70000/70000 [==============================] - 9s 131us/step - loss: 0.3154
Epoch 6/50
70000/70000 [==============================] - 15s 220us/step - loss: 0.3136
Epoch 7/50
70000/70000 [==============================] - 13s 189us/step - loss: 0.3114
Epoch 8/50
70000/70000 [==============================] - 10s 147us/step - loss: 0.3113
Epoch 9/50
70000/70000 [==============================] - 10s 144us/step - loss: 0.3096
Epoch 10/50
70000/70000 [==============================] - 10s 143us/step - loss: 0.3099
Epoch 11/50
70000/70000 [==============================] - 10s 143us/step - loss: 0.3095
Epoch 12/50
70000/70000 [==========

In [96]:
y_test_pred = model.predict({
    'main_input': pd.concat((test.drop(['ID_CORRELATIVO'], axis=1), 
                             test_extra), axis=1),
    'ensemble_input': ensemble_test.drop('ID_CORRELATIVO', axis=1)})

In [99]:
submission = pd.DataFrame()
submission['ID_CORRELATIVO'] = test['ID_CORRELATIVO']
submission['ATTRITION'] = y_test_pred[:,0]

In [100]:
submission.head()

,ID_CORRELATIVO,ATTRITION
0,47411,0.334118
1,39861,0.223711
2,38898,0.023750
3,50927,0.019959
4,32969,0.421899


In [101]:
submission.to_csv('_data/submission14_StackNet1.csv', index=False)

## New Architecture

In [102]:
def create_model():
    main_input = Input((train.shape[1] + train_extra.shape[1] - 2,), name='main_input')
    layer1A = Dense(100)(main_input)
    layer1A = Activation('relu')(layer1A)
    layer1A = BatchNormalization()(layer1A)
    layer1A = Dropout(0.2)(layer1A)
    
    layer1A = Dense(10)(layer1A)
    layer1A = Activation('relu')(layer1A)
    layer1A = BatchNormalization()(layer1A)
    layer1A = Dropout(0.3)(layer1A)
    
    ensemble_input = Input((ensemble_train.shape[1] - 1,), name='ensemble_input')
    layer1B = Dense(12)(ensemble_input)
    layer1B = Activation('relu')(layer1B)
    layer1B = BatchNormalization()(layer1B)
    layer1B = Dropout(0.2)(layer1B)
    
    x = concatenate([layer1A, layer1B])
    
    x = Dense(100)(x)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    
    main_output = Dense(1, activation='sigmoid', name='main_output')(x)
    
    model = Model(inputs=[main_input, ensemble_input], outputs=[main_output])
    
    adam = Adam()
    model.compile(loss='binary_crossentropy', optimizer=adam)
    return model

In [103]:
stacknet = create_model()

In [104]:
stacknet.fit({
    'main_input': pd.concat((train.drop(['ID_CORRELATIVO', 'ATTRITION'], axis=1), 
                             train_extra), axis=1),
    'ensemble_input': ensemble_train.drop('ID_CORRELATIVO', axis=1)
    }, train['ATTRITION'], batch_size=64, epochs=50)

Epoch 1/50
70000/70000 [==============================] - 8s 109us/step - loss: 0.4572
Epoch 2/50
70000/70000 [==============================] - 7s 107us/step - loss: 0.3559
Epoch 3/50
70000/70000 [==============================] - 8s 108us/step - loss: 0.3228
Epoch 4/50
70000/70000 [==============================] - 8s 112us/step - loss: 0.3146
Epoch 5/50
70000/70000 [==============================] - 7s 96us/step - loss: 0.3103
Epoch 6/50
70000/70000 [==============================] - 8s 118us/step - loss: 0.3092
Epoch 7/50
70000/70000 [==============================] - 7s 104us/step - loss: 0.3076
Epoch 8/50
70000/70000 [==============================] - 9s 125us/step - loss: 0.3078
Epoch 9/50
70000/70000 [==============================] - 10s 138us/step - loss: 0.3056
Epoch 10/50
70000/70000 [==============================] - 8s 111us/step - loss: 0.3063
Epoch 11/50
70000/70000 [==============================] - 8s 114us/step - loss: 0.3045
Epoch 12/50
70000/70000 [================

In [109]:
stacknet.fit({
    'main_input': pd.concat((train.drop(['ID_CORRELATIVO', 'ATTRITION'], axis=1), 
                             train_extra), axis=1),
    'ensemble_input': ensemble_train.drop('ID_CORRELATIVO', axis=1)
    }, train['ATTRITION'], batch_size=64, epochs=10)

Epoch 1/10
70000/70000 [==============================] - 6s 92us/step - loss: 0.3009
Epoch 2/10
70000/70000 [==============================] - 8s 109us/step - loss: 0.3006
Epoch 3/10
70000/70000 [==============================] - 10s 137us/step - loss: 0.3021
Epoch 4/10
70000/70000 [==============================] - 10s 138us/step - loss: 0.3008
Epoch 5/10
70000/70000 [==============================] - 11s 155us/step - loss: 0.3009
Epoch 6/10
70000/70000 [==============================] - 8s 114us/step - loss: 0.3003
Epoch 7/10
70000/70000 [==============================] - 8s 116us/step - loss: 0.3008
Epoch 8/10
70000/70000 [==============================] - 8s 116us/step - loss: 0.3003
Epoch 9/10
70000/70000 [==============================] - 9s 134us/step - loss: 0.3013
Epoch 10/10
70000/70000 [==============================] - 9s 126us/step - loss: 0.3002


In [110]:
stacknet.fit({
    'main_input': pd.concat((train.drop(['ID_CORRELATIVO', 'ATTRITION'], axis=1), 
                             train_extra), axis=1),
    'ensemble_input': ensemble_train.drop('ID_CORRELATIVO', axis=1)
    }, train['ATTRITION'], batch_size=64, epochs=10)

Epoch 1/10
70000/70000 [==============================] - 6s 92us/step - loss: 0.3009
Epoch 2/10
70000/70000 [==============================] - 6s 92us/step - loss: 0.3010
Epoch 3/10
70000/70000 [==============================] - 6s 92us/step - loss: 0.3004
Epoch 4/10
70000/70000 [==============================] - 8s 108us/step - loss: 0.3008
Epoch 5/10
70000/70000 [==============================] - 8s 112us/step - loss: 0.3003
Epoch 6/10
70000/70000 [==============================] - 8s 115us/step - loss: 0.3006
Epoch 7/10
70000/70000 [==============================] - 9s 136us/step - loss: 0.3002
Epoch 8/10
70000/70000 [==============================] - 9s 129us/step - loss: 0.3012
Epoch 9/10
70000/70000 [==============================] - 9s 125us/step - loss: 0.3006
Epoch 10/10
70000/70000 [==============================] - 9s 127us/step - loss: 0.3009


In [111]:
y_test_pred = stacknet.predict({
    'main_input': pd.concat((test.drop(['ID_CORRELATIVO'], axis=1), 
                             test_extra), axis=1),
    'ensemble_input': ensemble_test.drop('ID_CORRELATIVO', axis=1)})

In [112]:
submission = pd.DataFrame()
submission['ID_CORRELATIVO'] = test['ID_CORRELATIVO']
submission['ATTRITION'] = y_test_pred[:,0]

In [107]:
submission.head()

,ID_CORRELATIVO,ATTRITION
0,47411,0.251947
1,39861,0.254838
2,38898,0.026838
3,50927,0.024988
4,32969,0.296249


In [113]:
submission.head()

,ID_CORRELATIVO,ATTRITION
0,47411,0.244099
1,39861,0.246848
2,38898,0.016385
3,50927,0.019840
4,32969,0.300241


In [114]:
submission.to_csv('_data/submission15_StackNet_deep_02.csv', index=False)